In [1]:
# base
import pandas as pd
import numpy as np
from tqdm import tqdm
import lime
import math
import imblearn
import seaborn as sns
import shap
import copy
from matplotlib import pyplot as plt
import pycaret
from pycaret.classification import *
import pickle
from sklearn.model_selection import train_test_split
import time
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
import statistics

# warning
import warnings
warnings.filterwarnings("ignore")

In [2]:
selected_var_except_del=pd.read_csv("C:\\Users\\srija\\OneDrive\\Desktop\\MENTORSHIP\\amex kaggle competition\\selected var except del.csv")
selected_var_except_del

,selected except del
0,S_3
1,S_23
2,S_25
3,S_8
4,B_9
5,B_2
6,B_18
7,B_7
8,B_5
9,B_40


In [3]:
dev_full = pd.read_feather('D:\\AMEX\\train_data.ftr')
dev_full

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938477,0.001734,0.008728,1.006836,0.009224,0.124023,0.008774,0.004707,...,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936523,0.005775,0.004925,1.000977,0.006153,0.126709,0.000798,0.002714,...,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954102,0.091492,0.021652,1.009766,0.006817,0.123962,0.007599,0.009422,...,NaN,NaN,0.003269,0.007328,0.000427,NaN,0.003429,0.006985,0.002604,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960449,0.002455,0.013687,1.002930,0.001372,0.117188,0.000685,0.005531,...,NaN,NaN,0.006119,0.004517,0.003201,NaN,0.008423,0.006527,0.009598,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947266,0.002483,0.015190,1.000977,0.007607,0.117310,0.004654,0.009308,...,NaN,NaN,0.003672,0.004944,0.008888,NaN,0.001670,0.008125,0.009827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-11-05,0.979492,0.416016,0.020813,0.828125,0.003487,0.090759,0.005341,0.025146,...,NaN,NaN,0.006836,0.003679,0.000457,NaN,0.000906,0.001497,0.002775,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-12-23,0.984863,0.296631,0.007210,0.812500,0.005905,0.079895,0.002243,0.023697,...,NaN,NaN,0.003309,0.007095,0.007858,NaN,0.002777,0.008224,0.008858,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-01-06,0.982910,0.444092,0.013153,0.815430,0.003456,0.100525,0.002111,0.012344,...,NaN,NaN,0.009956,0.009995,0.001088,NaN,0.005692,0.006775,0.005566,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-02-06,0.969727,0.442627,0.009857,1.003906,0.005116,0.101807,0.009933,0.008575,...,NaN,NaN,0.005543,0.006565,0.009880,NaN,0.008125,0.001168,0.003983,0


In [4]:
cat_feat_full = ['D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [5]:
delinquent_feat = [i for i in dev_full.columns if i.startswith('D_') and i not in cat_feat_full]

In [6]:
len(delinquent_feat)

87

In [7]:
cols_to_read=['customer_ID','S_2']+list(selected_var_except_del['selected except del'])+delinquent_feat+['target']
cols_to_read

['customer_ID',
 'S_2',
 'S_3',
 'S_23',
 'S_25',
 'S_8',
 'B_9',
 'B_2',
 'B_18',
 'B_7',
 'B_5',
 'B_40',
 'B_33',
 'B_3',
 'B_8',
 'R_1',
 'R_10',
 'R_3',
 'R_2',
 'R_6',
 'R_27',
 'R_16',
 'R_7',
 'R_5',
 'P_2',
 'B_30',
 'D_120',
 'D_39',
 'D_41',
 'D_42',
 'D_43',
 'D_44',
 'D_45',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'D_50',
 'D_51',
 'D_52',
 'D_53',
 'D_54',
 'D_55',
 'D_56',
 'D_58',
 'D_59',
 'D_60',
 'D_61',
 'D_62',
 'D_65',
 'D_69',
 'D_70',
 'D_71',
 'D_72',
 'D_73',
 'D_74',
 'D_75',
 'D_76',
 'D_77',
 'D_78',
 'D_79',
 'D_80',
 'D_81',
 'D_82',
 'D_83',
 'D_84',
 'D_86',
 'D_87',
 'D_88',
 'D_89',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'D_96',
 'D_102',
 'D_103',
 'D_104',
 'D_105',
 'D_106',
 'D_107',
 'D_108',
 'D_109',
 'D_110',
 'D_111',
 'D_112',
 'D_113',
 'D_115',
 'D_118',
 'D_119',
 'D_121',
 'D_122',
 'D_123',
 'D_124',
 'D_125',
 'D_127',
 'D_128',
 'D_129',
 'D_130',
 'D_131',
 'D_132',
 'D_133',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_139',
 'D

In [8]:
dev = pd.read_feather('D:\\AMEX\\train_data.ftr',columns=cols_to_read)
dev

,customer_ID,S_2,S_3,S_23,S_25,S_8,B_9,B_2,B_18,B_7,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.124023,0.135620,0.974609,0.922852,0.008209,1.006836,0.652832,0.059418,...,NaN,NaN,0.002426,0.003706,0.003819,NaN,0.000569,0.000610,0.002674,0
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.126709,0.136353,0.975586,0.919434,0.008369,1.000977,0.646973,0.057739,...,NaN,NaN,0.003956,0.003166,0.005032,NaN,0.009575,0.005493,0.009216,0
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.123962,0.134888,0.974121,1.001953,0.009354,1.009766,0.645996,0.056641,...,NaN,NaN,0.003269,0.007328,0.000427,NaN,0.003429,0.006985,0.002604,0
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.117188,0.140015,0.975586,0.704102,0.006783,1.002930,0.654297,0.049255,...,NaN,NaN,0.006119,0.004517,0.003201,NaN,0.008423,0.006527,0.009598,0
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.117310,0.131592,0.978027,0.916992,0.000519,1.000977,0.649902,0.048920,...,NaN,NaN,0.003672,0.004944,0.008888,NaN,0.001670,0.008125,0.009827,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531446,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-11-05,0.090759,0.132080,0.970703,0.705078,0.013779,0.828125,0.442871,0.018127,...,NaN,NaN,0.006836,0.003679,0.000457,NaN,0.000906,0.001497,0.002775,0
5531447,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2017-12-23,0.079895,0.137573,0.974609,0.705078,0.005646,0.812500,1.006836,0.021851,...,NaN,NaN,0.003309,0.007095,0.007858,NaN,0.002777,0.008224,0.008858,0
5531448,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-01-06,0.100525,0.138672,0.972656,0.326904,0.004089,0.815430,1.001953,0.016144,...,NaN,NaN,0.009956,0.009995,0.001088,NaN,0.005692,0.006775,0.005566,0
5531449,fffff1d38b785cef84adeace64f8f83db3a0c31e8d92ea...,2018-02-06,0.101807,0.131958,0.976074,0.327881,0.006626,1.003906,1.008789,0.014801,...,NaN,NaN,0.005543,0.006565,0.009880,NaN,0.008125,0.001168,0.003983,0


In [9]:
cat_feat = ['B_30', 'D_120']
drop_feat = ['customer_ID', 'S_2']
num_feat = sorted([f for f in dev.columns if f not in cat_feat + drop_feat + ['target']])
spend_feat      = [i for i in num_feat if i.startswith('S_')]
payment_feat    = [i for i in num_feat if i.startswith('P_')]
balance_feat    = [i for i in num_feat if i.startswith('B_')]
risk_feat       = [i for i in num_feat if i.startswith('R_')]

In [10]:
features=[f for f in dev.columns if f not in drop_feat + ['target']]

In [11]:
num_agg1 = dev.groupby("customer_ID")[num_feat].agg(['mean', 'std', 'min', 'max'])
num_agg1.columns = ['_'.join(x) for x in num_agg1.columns]
num_agg1.reset_index(level=0, inplace=True)
num_agg1

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,S_25_min,S_25_max,S_3_mean,S_3_std,S_3_min,S_3_max,S_8_mean,S_8_std,S_8_min,S_8_max
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.184871,0.645996,1.007812,1.004883,0.003248,1.000000,1.009766,0.006454,...,0.969727,0.979004,0.113220,0.011665,0.098877,0.135010,0.796875,0.136276,0.488281,1.001953
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,0.003189,1.000000,1.009766,0.991211,0.051519,0.819824,1.008789,0.005665,...,0.970215,0.979004,0.120605,0.023842,0.089783,0.165527,0.411133,0.243573,0.002707,0.763672
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.137020,0.689453,1.008789,0.815430,0.003485,0.811035,0.819824,0.005493,...,0.969238,0.979004,NaN,NaN,NaN,NaN,0.006432,0.002996,0.000590,0.009521
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.183675,0.543945,1.008789,0.955078,0.080903,0.812012,1.009766,0.006424,...,0.969727,0.978516,0.247803,0.095130,0.149170,0.407471,0.308594,0.129054,0.167847,0.481201
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.066477,0.528809,0.692871,0.814453,0.003130,0.810547,0.819824,0.005089,...,0.969238,0.978027,0.173096,0.004681,0.166138,0.176392,0.053467,0.122191,0.000093,0.330078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,0.002819,1.000977,1.008789,1.004883,0.003046,1.000977,1.009766,0.005814,...,0.971191,0.978027,0.125977,0.012347,0.103760,0.140137,0.513672,0.157533,0.315186,0.759766
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.014654,0.098267,0.154907,0.042358,0.007478,0.030441,0.055664,0.271973,...,0.970703,0.978516,0.273682,0.071251,0.194702,0.334473,0.050598,0.166301,0.000219,0.604004
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.248264,0.311523,1.009766,0.854492,0.366320,0.004150,1.009766,0.057556,...,0.970215,0.979004,0.084351,0.008972,0.066650,0.096558,0.690430,0.111980,0.466309,0.924316
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.093677,0.420410,0.653809,0.675781,0.345028,0.182861,1.005859,0.071106,...,0.969238,0.979004,0.357910,0.130224,0.182007,0.609375,0.076782,0.118591,0.000257,0.326416


In [12]:
num_feat_withid=['customer_ID']
for i in num_feat:
    num_feat_withid.append(i)
num_feat_withid 

['customer_ID',
 'B_18',
 'B_2',
 'B_3',
 'B_33',
 'B_40',
 'B_5',
 'B_7',
 'B_8',
 'B_9',
 'D_102',
 'D_103',
 'D_104',
 'D_105',
 'D_106',
 'D_107',
 'D_108',
 'D_109',
 'D_110',
 'D_111',
 'D_112',
 'D_113',
 'D_115',
 'D_118',
 'D_119',
 'D_121',
 'D_122',
 'D_123',
 'D_124',
 'D_125',
 'D_127',
 'D_128',
 'D_129',
 'D_130',
 'D_131',
 'D_132',
 'D_133',
 'D_134',
 'D_135',
 'D_136',
 'D_137',
 'D_138',
 'D_139',
 'D_140',
 'D_141',
 'D_142',
 'D_143',
 'D_144',
 'D_145',
 'D_39',
 'D_41',
 'D_42',
 'D_43',
 'D_44',
 'D_45',
 'D_46',
 'D_47',
 'D_48',
 'D_49',
 'D_50',
 'D_51',
 'D_52',
 'D_53',
 'D_54',
 'D_55',
 'D_56',
 'D_58',
 'D_59',
 'D_60',
 'D_61',
 'D_62',
 'D_65',
 'D_69',
 'D_70',
 'D_71',
 'D_72',
 'D_73',
 'D_74',
 'D_75',
 'D_76',
 'D_77',
 'D_78',
 'D_79',
 'D_80',
 'D_81',
 'D_82',
 'D_83',
 'D_84',
 'D_86',
 'D_87',
 'D_88',
 'D_89',
 'D_91',
 'D_92',
 'D_93',
 'D_94',
 'D_96',
 'P_2',
 'R_1',
 'R_10',
 'R_16',
 'R_2',
 'R_27',
 'R_3',
 'R_5',
 'R_6',
 'R_7',
 'S_

In [13]:
col_name2=['customer_ID']
for i in num_feat:
    col_name2.append(i+str('_last obs'))
col_name2

['customer_ID',
 'B_18_last obs',
 'B_2_last obs',
 'B_3_last obs',
 'B_33_last obs',
 'B_40_last obs',
 'B_5_last obs',
 'B_7_last obs',
 'B_8_last obs',
 'B_9_last obs',
 'D_102_last obs',
 'D_103_last obs',
 'D_104_last obs',
 'D_105_last obs',
 'D_106_last obs',
 'D_107_last obs',
 'D_108_last obs',
 'D_109_last obs',
 'D_110_last obs',
 'D_111_last obs',
 'D_112_last obs',
 'D_113_last obs',
 'D_115_last obs',
 'D_118_last obs',
 'D_119_last obs',
 'D_121_last obs',
 'D_122_last obs',
 'D_123_last obs',
 'D_124_last obs',
 'D_125_last obs',
 'D_127_last obs',
 'D_128_last obs',
 'D_129_last obs',
 'D_130_last obs',
 'D_131_last obs',
 'D_132_last obs',
 'D_133_last obs',
 'D_134_last obs',
 'D_135_last obs',
 'D_136_last obs',
 'D_137_last obs',
 'D_138_last obs',
 'D_139_last obs',
 'D_140_last obs',
 'D_141_last obs',
 'D_142_last obs',
 'D_143_last obs',
 'D_144_last obs',
 'D_145_last obs',
 'D_39_last obs',
 'D_41_last obs',
 'D_42_last obs',
 'D_43_last obs',
 'D_44_last obs

In [14]:
dev_last=dev.groupby("customer_ID").tail(1)[num_feat_withid]
dev_last.columns=col_name2
dev_last.fillna(value=0,inplace=True)
dev_last

,customer_ID,B_18_last obs,B_2_last obs,B_3_last obs,B_33_last obs,B_40_last obs,B_5_last obs,B_7_last obs,B_8_last obs,B_9_last obs,...,R_2_last obs,R_27_last obs,R_3_last obs,R_5_last obs,R_6_last obs,R_7_last obs,S_23_last obs,S_25_last obs,S_3_last obs,S_8_last obs
12,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.007812,1.007812,0.007175,1.007812,0.100464,0.231689,0.058411,0.002928,0.009537,...,0.008308,1.005859,0.000085,0.009445,0.003944,0.009163,0.131836,0.972168,0.135010,0.488281
25,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.003906,1.003906,0.005070,1.000977,0.019806,0.026993,0.028412,0.000974,0.012924,...,0.004978,1.007812,0.102051,0.005878,0.007744,0.008255,0.132812,0.977539,0.165527,0.406494
38,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,1.003906,0.812500,0.007195,1.000977,0.024902,0.001557,0.026978,0.000247,0.009392,...,0.001687,1.002930,0.006264,0.003000,0.008575,0.006741,0.132690,0.972168,0.000000,0.009186
51,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,1.006836,1.005859,0.009941,1.009766,0.031021,0.118835,0.011971,1.005859,0.020523,...,0.004238,1.004883,0.204346,0.001347,0.000952,0.006096,0.132812,0.974121,0.287842,0.170532
64,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.531250,0.815918,0.005527,1.007812,0.116821,0.004856,0.159790,1.004883,0.004028,...,0.001991,1.006836,0.000050,0.008858,0.003574,0.006454,0.132080,0.971680,0.000000,0.008591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5531398,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.001953,1.009766,0.005894,1.005859,0.008415,0.040527,0.029922,1.005859,0.170288,...,0.007812,1.002930,0.709961,0.001793,0.004341,0.009193,0.136597,0.974609,0.128662,0.494385
5531411,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.098267,0.055664,0.233032,0.000740,0.203735,0.018677,0.272217,0.004753,0.143188,...,0.009918,1.007812,0.109131,0.008003,0.009163,0.006332,0.139038,0.976562,0.000000,0.002062
5531424,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,1.002930,1.006836,0.006313,1.000977,0.051025,0.019531,0.048523,1.001953,0.046448,...,0.005898,1.008789,0.008339,0.006081,0.003065,0.005630,0.140625,0.972656,0.066650,0.855469
5531437,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.465088,0.714355,0.050049,0.008911,0.329346,0.020523,0.302490,1.004883,0.016495,...,0.005466,1.003906,0.000832,0.000409,0.003014,0.001539,0.131836,0.979004,0.408936,0.001601


In [15]:
col_name=['customer_ID']
for i in num_feat:
    col_name.append(i+str('_last3_avg'))
col_name

['customer_ID',
 'B_18_last3_avg',
 'B_2_last3_avg',
 'B_3_last3_avg',
 'B_33_last3_avg',
 'B_40_last3_avg',
 'B_5_last3_avg',
 'B_7_last3_avg',
 'B_8_last3_avg',
 'B_9_last3_avg',
 'D_102_last3_avg',
 'D_103_last3_avg',
 'D_104_last3_avg',
 'D_105_last3_avg',
 'D_106_last3_avg',
 'D_107_last3_avg',
 'D_108_last3_avg',
 'D_109_last3_avg',
 'D_110_last3_avg',
 'D_111_last3_avg',
 'D_112_last3_avg',
 'D_113_last3_avg',
 'D_115_last3_avg',
 'D_118_last3_avg',
 'D_119_last3_avg',
 'D_121_last3_avg',
 'D_122_last3_avg',
 'D_123_last3_avg',
 'D_124_last3_avg',
 'D_125_last3_avg',
 'D_127_last3_avg',
 'D_128_last3_avg',
 'D_129_last3_avg',
 'D_130_last3_avg',
 'D_131_last3_avg',
 'D_132_last3_avg',
 'D_133_last3_avg',
 'D_134_last3_avg',
 'D_135_last3_avg',
 'D_136_last3_avg',
 'D_137_last3_avg',
 'D_138_last3_avg',
 'D_139_last3_avg',
 'D_140_last3_avg',
 'D_141_last3_avg',
 'D_142_last3_avg',
 'D_143_last3_avg',
 'D_144_last3_avg',
 'D_145_last3_avg',
 'D_39_last3_avg',
 'D_41_last3_avg',
 

In [16]:
dev_last3_avg = dev.groupby("customer_ID").tail(3).groupby("customer_ID")[num_feat].agg(['mean'])
dev_last3_avg.reset_index(level=0, inplace=True)
dev_last3_avg.columns=col_name
dev_last3_avg.fillna(value=0,inplace=True)
dev_last3_avg

,customer_ID,B_18_last3_avg,B_2_last3_avg,B_3_last3_avg,B_33_last3_avg,B_40_last3_avg,B_5_last3_avg,B_7_last3_avg,B_8_last3_avg,B_9_last3_avg,...,R_2_last3_avg,R_27_last3_avg,R_3_last3_avg,R_5_last3_avg,R_6_last3_avg,R_7_last3_avg,S_23_last3_avg,S_25_last3_avg,S_3_last3_avg,S_8_last3_avg
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,1.006836,1.006836,0.005955,1.005859,0.068359,0.149902,0.042908,0.004009,0.005772,...,0.004684,1.005859,0.006153,0.004440,0.005276,0.006279,0.134277,0.975098,0.112305,0.667969
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.001953,1.003906,0.005535,1.002930,0.019424,0.016663,0.028152,0.002504,0.011314,...,0.005398,1.006836,0.103821,0.005157,0.005196,0.007919,0.136475,0.976074,0.151733,0.352295
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,1.006836,0.814941,0.005135,1.003906,0.029099,0.000664,0.031494,0.003859,0.004860,...,0.006245,1.002930,0.006832,0.006489,0.006195,0.005226,0.133789,0.972168,0.000000,0.009216
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.853516,0.994629,0.007717,1.005859,0.084351,0.091553,0.019577,1.006836,0.040833,...,0.007172,1.006836,0.070496,0.001671,0.004627,0.003633,0.135498,0.974609,0.302002,0.221680
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.531250,0.814453,0.003180,1.004883,0.130005,0.004089,0.163086,1.004883,0.007534,...,0.003832,1.004883,0.003218,0.007614,0.004467,0.005623,0.133667,0.972656,0.000000,0.004776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.003906,1.006836,0.007744,1.006836,0.015320,0.172485,0.018494,1.004883,0.095459,...,0.008247,1.004883,0.439697,0.004734,0.006508,0.006912,0.138428,0.974121,0.133057,0.492920
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.131104,0.051056,0.240356,0.006596,0.198730,0.018036,0.247314,0.005501,0.150879,...,0.005421,1.006836,0.073669,0.005798,0.007057,0.005451,0.135742,0.974609,0.000000,0.001610
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,1.005859,1.002930,0.003960,1.000977,0.039001,0.039246,0.039825,1.001953,0.048370,...,0.006100,1.009766,0.037445,0.005764,0.004448,0.005405,0.137695,0.974609,0.070923,0.675293
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.457764,0.376953,0.073669,0.005074,0.418945,0.012161,0.282959,1.003906,0.014397,...,0.004517,1.006836,0.003464,0.004715,0.005169,0.004791,0.135132,0.973633,0.481445,0.110168


In [17]:
num_agg2=pd.merge(num_agg1,dev_last, left_on = 'customer_ID', right_on = 'customer_ID',how = 'outer')
num_agg2

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_last obs,R_27_last obs,R_3_last obs,R_5_last obs,R_6_last obs,R_7_last obs,S_23_last obs,S_25_last obs,S_3_last obs,S_8_last obs
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.184871,0.645996,1.007812,1.004883,0.003248,1.000000,1.009766,0.006454,...,0.008308,1.005859,0.000085,0.009445,0.003944,0.009163,0.131836,0.972168,0.135010,0.488281
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,0.003189,1.000000,1.009766,0.991211,0.051519,0.819824,1.008789,0.005665,...,0.004978,1.007812,0.102051,0.005878,0.007744,0.008255,0.132812,0.977539,0.165527,0.406494
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.137020,0.689453,1.008789,0.815430,0.003485,0.811035,0.819824,0.005493,...,0.001687,1.002930,0.006264,0.003000,0.008575,0.006741,0.132690,0.972168,0.000000,0.009186
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.183675,0.543945,1.008789,0.955078,0.080903,0.812012,1.009766,0.006424,...,0.004238,1.004883,0.204346,0.001347,0.000952,0.006096,0.132812,0.974121,0.287842,0.170532
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.066477,0.528809,0.692871,0.814453,0.003130,0.810547,0.819824,0.005089,...,0.001991,1.006836,0.000050,0.008858,0.003574,0.006454,0.132080,0.971680,0.000000,0.008591
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,0.002819,1.000977,1.008789,1.004883,0.003046,1.000977,1.009766,0.005814,...,0.007812,1.002930,0.709961,0.001793,0.004341,0.009193,0.136597,0.974609,0.128662,0.494385
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.014654,0.098267,0.154907,0.042358,0.007478,0.030441,0.055664,0.271973,...,0.009918,1.007812,0.109131,0.008003,0.009163,0.006332,0.139038,0.976562,0.000000,0.002062
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.248264,0.311523,1.009766,0.854492,0.366320,0.004150,1.009766,0.057556,...,0.005898,1.008789,0.008339,0.006081,0.003065,0.005630,0.140625,0.972656,0.066650,0.855469
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.093677,0.420410,0.653809,0.675781,0.345028,0.182861,1.005859,0.071106,...,0.005466,1.003906,0.000832,0.000409,0.003014,0.001539,0.131836,0.979004,0.408936,0.001601


In [18]:
for i in num_feat:
    last=i+str('_last obs')
    mean=i+str('_mean')
    std=i+str('_std')
    flag_last=i+str('_flag_last')
    num_agg2[flag_last]=num_agg2[last]>num_agg2[mean]+3*num_agg2[std] 
    num_agg2[flag_last]=num_agg2[flag_last].astype(int)
num_agg2    

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_flag_last,R_27_flag_last,R_3_flag_last,R_5_flag_last,R_6_flag_last,R_7_flag_last,S_23_flag_last,S_25_flag_last,S_3_flag_last,S_8_flag_last
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.184871,0.645996,1.007812,1.004883,0.003248,1.000000,1.009766,0.006454,...,0,0,0,0,0,0,0,0,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,0.003189,1.000000,1.009766,0.991211,0.051519,0.819824,1.008789,0.005665,...,0,0,0,0,0,0,0,0,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.137020,0.689453,1.008789,0.815430,0.003485,0.811035,0.819824,0.005493,...,0,0,0,0,0,0,0,0,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.183675,0.543945,1.008789,0.955078,0.080903,0.812012,1.009766,0.006424,...,0,0,0,0,0,0,0,0,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.066477,0.528809,0.692871,0.814453,0.003130,0.810547,0.819824,0.005089,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,0.002819,1.000977,1.008789,1.004883,0.003046,1.000977,1.009766,0.005814,...,0,0,0,0,0,0,0,0,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.014654,0.098267,0.154907,0.042358,0.007478,0.030441,0.055664,0.271973,...,0,0,0,0,0,0,0,0,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.248264,0.311523,1.009766,0.854492,0.366320,0.004150,1.009766,0.057556,...,0,0,0,0,0,0,0,0,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.093677,0.420410,0.653809,0.675781,0.345028,0.182861,1.005859,0.071106,...,0,0,0,0,0,0,0,0,0,0


In [19]:
num_agg3=pd.merge(num_agg2,dev_last3_avg, left_on = 'customer_ID', right_on = 'customer_ID',how = 'outer')
num_agg3

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_last3_avg,R_27_last3_avg,R_3_last3_avg,R_5_last3_avg,R_6_last3_avg,R_7_last3_avg,S_23_last3_avg,S_25_last3_avg,S_3_last3_avg,S_8_last3_avg
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.184871,0.645996,1.007812,1.004883,0.003248,1.000000,1.009766,0.006454,...,0.004684,1.005859,0.006153,0.004440,0.005276,0.006279,0.134277,0.975098,0.112305,0.667969
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,0.003189,1.000000,1.009766,0.991211,0.051519,0.819824,1.008789,0.005665,...,0.005398,1.006836,0.103821,0.005157,0.005196,0.007919,0.136475,0.976074,0.151733,0.352295
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.137020,0.689453,1.008789,0.815430,0.003485,0.811035,0.819824,0.005493,...,0.006245,1.002930,0.006832,0.006489,0.006195,0.005226,0.133789,0.972168,0.000000,0.009216
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.183675,0.543945,1.008789,0.955078,0.080903,0.812012,1.009766,0.006424,...,0.007172,1.006836,0.070496,0.001671,0.004627,0.003633,0.135498,0.974609,0.302002,0.221680
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.066477,0.528809,0.692871,0.814453,0.003130,0.810547,0.819824,0.005089,...,0.003832,1.004883,0.003218,0.007614,0.004467,0.005623,0.133667,0.972656,0.000000,0.004776
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,0.002819,1.000977,1.008789,1.004883,0.003046,1.000977,1.009766,0.005814,...,0.008247,1.004883,0.439697,0.004734,0.006508,0.006912,0.138428,0.974121,0.133057,0.492920
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.014654,0.098267,0.154907,0.042358,0.007478,0.030441,0.055664,0.271973,...,0.005421,1.006836,0.073669,0.005798,0.007057,0.005451,0.135742,0.974609,0.000000,0.001610
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.248264,0.311523,1.009766,0.854492,0.366320,0.004150,1.009766,0.057556,...,0.006100,1.009766,0.037445,0.005764,0.004448,0.005405,0.137695,0.974609,0.070923,0.675293
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.093677,0.420410,0.653809,0.675781,0.345028,0.182861,1.005859,0.071106,...,0.004517,1.006836,0.003464,0.004715,0.005169,0.004791,0.135132,0.973633,0.481445,0.110168


In [20]:
for i in num_feat:
    last3=i+str('_last3_avg')
    mean=i+str('_mean')
    std=i+str('_std')
    flag_last3=i+str('_flag_last3')
    num_agg3[flag_last3]=num_agg3[last3]>num_agg3[mean]+3*num_agg3[std] 
    num_agg3[flag_last3]=num_agg3[flag_last3].astype(int)
num_agg3    

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,R_2_flag_last3,R_27_flag_last3,R_3_flag_last3,R_5_flag_last3,R_6_flag_last3,R_7_flag_last3,S_23_flag_last3,S_25_flag_last3,S_3_flag_last3,S_8_flag_last3
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.184871,0.645996,1.007812,1.004883,0.003248,1.000000,1.009766,0.006454,...,0,0,0,0,0,0,0,0,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,0.003189,1.000000,1.009766,0.991211,0.051519,0.819824,1.008789,0.005665,...,0,0,0,0,0,0,0,0,0,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.137020,0.689453,1.008789,0.815430,0.003485,0.811035,0.819824,0.005493,...,0,0,0,0,0,0,0,0,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.183675,0.543945,1.008789,0.955078,0.080903,0.812012,1.009766,0.006424,...,0,0,0,0,0,0,0,0,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.066477,0.528809,0.692871,0.814453,0.003130,0.810547,0.819824,0.005089,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,0.002819,1.000977,1.008789,1.004883,0.003046,1.000977,1.009766,0.005814,...,0,0,0,0,0,0,0,0,0,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.014654,0.098267,0.154907,0.042358,0.007478,0.030441,0.055664,0.271973,...,0,0,0,0,0,0,0,0,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.248264,0.311523,1.009766,0.854492,0.366320,0.004150,1.009766,0.057556,...,0,0,0,0,0,0,0,0,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.093677,0.420410,0.653809,0.675781,0.345028,0.182861,1.005859,0.071106,...,0,0,0,0,0,0,0,0,0,0


In [12]:
cat_agg = dev.groupby("customer_ID")[cat_feat].agg(['count', 'last', 'nunique'])
cat_agg.columns = ['_'.join(x) for x in cat_agg.columns]
cat_agg=pd.get_dummies(cat_agg,columns=list(cat_agg.columns))  
cat_agg.reset_index(level=0, inplace=True)
cat_agg

,customer_ID,B_30_count_0,B_30_count_1,B_30_count_2,B_30_count_3,B_30_count_4,B_30_count_5,B_30_count_6,B_30_count_7,B_30_count_8,...,D_120_count_9,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,1
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,1,0


In [22]:
final_agg=pd.merge(num_agg3,cat_agg, left_on = 'customer_ID', right_on = 'customer_ID',how = 'outer')
final_agg

,customer_ID,B_18_mean,B_18_std,B_18_min,B_18_max,B_2_mean,B_2_std,B_2_min,B_2_max,B_3_mean,...,D_120_count_9,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.184871,0.645996,1.007812,1.004883,0.003248,1.000000,1.009766,0.006454,...,0,0,0,0,1,1,0,0,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,0.003189,1.000000,1.009766,0.991211,0.051519,0.819824,1.008789,0.005665,...,0,0,0,0,1,1,0,0,0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.137020,0.689453,1.008789,0.815430,0.003485,0.811035,0.819824,0.005493,...,0,0,0,0,1,1,0,0,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.183675,0.543945,1.008789,0.955078,0.080903,0.812012,1.009766,0.006424,...,0,0,0,0,1,1,0,0,1,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.066477,0.528809,0.692871,0.814453,0.003130,0.810547,0.819824,0.005089,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,0.002819,1.000977,1.008789,1.004883,0.003046,1.000977,1.009766,0.005814,...,0,0,0,0,1,0,1,0,0,1
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.014654,0.098267,0.154907,0.042358,0.007478,0.030441,0.055664,0.271973,...,0,0,0,0,1,1,0,0,1,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.248264,0.311523,1.009766,0.854492,0.366320,0.004150,1.009766,0.057556,...,0,0,0,0,1,1,0,0,1,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.093677,0.420410,0.653809,0.675781,0.345028,0.182861,1.005859,0.071106,...,0,0,0,0,1,1,0,0,1,0


In [23]:
#dropping the std features
for i in num_feat:
    std=i+str('_std')
    final_agg.drop(columns=std,inplace=True)
final_agg

,customer_ID,B_18_mean,B_18_min,B_18_max,B_2_mean,B_2_min,B_2_max,B_3_mean,B_3_min,B_3_max,...,D_120_count_9,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.645996,1.007812,1.004883,1.000000,1.009766,0.006454,0.000783,0.009865,...,0,0,0,0,1,1,0,0,1,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,1.000000,1.009766,0.991211,0.819824,1.008789,0.005665,0.000861,0.012863,...,0,0,0,0,1,1,0,0,0,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.689453,1.008789,0.815430,0.811035,0.819824,0.005493,0.000627,0.009384,...,0,0,0,0,1,1,0,0,1,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.543945,1.008789,0.955078,0.812012,1.009766,0.006424,0.000053,0.010925,...,0,0,0,0,1,1,0,0,1,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.528809,0.692871,0.814453,0.810547,0.819824,0.005089,0.000049,0.009689,...,0,0,0,0,1,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,1.000977,1.008789,1.004883,1.000977,1.009766,0.005814,0.000242,0.010162,...,0,0,0,0,1,0,1,0,0,1
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.098267,0.154907,0.042358,0.030441,0.055664,0.271973,0.231812,0.303467,...,0,0,0,0,1,1,0,0,1,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.311523,1.009766,0.854492,0.004150,1.009766,0.057556,0.000675,0.299072,...,0,0,0,0,1,1,0,0,1,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.420410,0.653809,0.675781,0.182861,1.005859,0.071106,0.039642,0.108521,...,0,0,0,0,1,1,0,0,1,0


In [24]:
dev_target = dev.groupby("customer_ID")['target'].agg(['mean'])
dev_target.columns = ['target']
dev_target = dev_target.target.astype(int)

In [25]:
train = final_agg.merge(dev_target, on='customer_ID', how='left')
train

,customer_ID,B_18_mean,B_18_min,B_18_max,B_2_mean,B_2_min,B_2_max,B_3_mean,B_3_min,B_3_max,...,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.842773,0.645996,1.007812,1.004883,1.000000,1.009766,0.006454,0.000783,0.009865,...,0,0,0,1,1,0,0,1,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.004883,1.000000,1.009766,0.991211,0.819824,1.008789,0.005665,0.000861,0.012863,...,0,0,0,1,1,0,0,0,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.933105,0.689453,1.008789,0.815430,0.811035,0.819824,0.005493,0.000627,0.009384,...,0,0,0,1,1,0,0,1,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.910645,0.543945,1.008789,0.955078,0.812012,1.009766,0.006424,0.000053,0.010925,...,0,0,0,1,1,0,0,1,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.609863,0.528809,0.692871,0.814453,0.810547,0.819824,0.005089,0.000049,0.009689,...,0,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.004883,1.000977,1.008789,1.004883,1.000977,1.009766,0.005814,0.000242,0.010162,...,0,0,0,1,0,1,0,0,1,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.146118,0.098267,0.154907,0.042358,0.030441,0.055664,0.271973,0.231812,0.303467,...,0,0,0,1,1,0,0,1,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.904297,0.311523,1.009766,0.854492,0.004150,1.009766,0.057556,0.000675,0.299072,...,0,0,0,1,1,0,0,1,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.580078,0.420410,0.653809,0.675781,0.182861,1.005859,0.071106,0.039642,0.108521,...,0,0,0,1,1,0,0,1,0,1


In [26]:
train.to_csv('D:\\AMEX\\final train to model_del all.csv',index=False)

In [11]:
train=pd.read_csv('D:\\AMEX\\final train to model_del all.csv')
train

,customer_ID,B_18_mean,B_18_min,B_18_max,B_2_mean,B_2_min,B_2_max,B_3_mean,B_3_min,B_3_max,...,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.8430,0.64600,1.0080,1.00500,1.00000,1.01000,0.006454,0.000783,0.009865,...,0,0,0,1,1,0,0,1,0,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,1.0050,1.00000,1.0100,0.99100,0.82000,1.00900,0.005665,0.000861,0.012860,...,0,0,0,1,1,0,0,0,1,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.9330,0.68950,1.0090,0.81540,0.81100,0.82000,0.005493,0.000627,0.009384,...,0,0,0,1,1,0,0,1,0,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.9106,0.54400,1.0090,0.95500,0.81200,1.01000,0.006424,0.000053,0.010925,...,0,0,0,1,1,0,0,1,0,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.6100,0.52900,0.6930,0.81450,0.81050,0.82000,0.005090,0.000049,0.009690,...,0,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.0050,1.00100,1.0090,1.00500,1.00100,1.01000,0.005814,0.000242,0.010160,...,0,0,0,1,0,1,0,0,1,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.1461,0.09827,0.1549,0.04236,0.03044,0.05566,0.272000,0.231800,0.303500,...,0,0,0,1,1,0,0,1,0,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.9043,0.31150,1.0100,0.85450,0.00415,1.01000,0.057560,0.000675,0.299000,...,0,0,0,1,1,0,0,1,0,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.5800,0.42040,0.6540,0.67600,0.18290,1.00600,0.071100,0.039640,0.108500,...,0,0,0,1,1,0,0,1,0,1


In [13]:
a=list(cat_agg.columns)
a.append('target')
for i in num_feat:
    flag_last=i+str('_flag_last')
    flag_last3=i+str('_flag_last3')
    a.append(flag_last)
    a.append(flag_last3)
a    

['customer_ID',
 'B_30_count_0',
 'B_30_count_1',
 'B_30_count_2',
 'B_30_count_3',
 'B_30_count_4',
 'B_30_count_5',
 'B_30_count_6',
 'B_30_count_7',
 'B_30_count_8',
 'B_30_count_9',
 'B_30_count_10',
 'B_30_count_11',
 'B_30_count_12',
 'B_30_count_13',
 'B_30_last_0.0',
 'B_30_last_1.0',
 'B_30_last_2.0',
 'B_30_nunique_0',
 'B_30_nunique_1',
 'B_30_nunique_2',
 'B_30_nunique_3',
 'D_120_count_0',
 'D_120_count_1',
 'D_120_count_2',
 'D_120_count_3',
 'D_120_count_4',
 'D_120_count_5',
 'D_120_count_6',
 'D_120_count_7',
 'D_120_count_8',
 'D_120_count_9',
 'D_120_count_10',
 'D_120_count_11',
 'D_120_count_12',
 'D_120_count_13',
 'D_120_last_0.0',
 'D_120_last_1.0',
 'D_120_nunique_0',
 'D_120_nunique_1',
 'D_120_nunique_2',
 'target',
 'B_18_flag_last',
 'B_18_flag_last3',
 'B_2_flag_last',
 'B_2_flag_last3',
 'B_3_flag_last',
 'B_3_flag_last3',
 'B_33_flag_last',
 'B_33_flag_last3',
 'B_40_flag_last',
 'B_40_flag_last3',
 'B_5_flag_last',
 'B_5_flag_last3',
 'B_7_flag_last',
 

In [14]:
num_columns = sorted([f for f in train.columns if f not in a])
num_columns

['B_18_last obs',
 'B_18_last3_avg',
 'B_18_max',
 'B_18_mean',
 'B_18_min',
 'B_2_last obs',
 'B_2_last3_avg',
 'B_2_max',
 'B_2_mean',
 'B_2_min',
 'B_33_last obs',
 'B_33_last3_avg',
 'B_33_max',
 'B_33_mean',
 'B_33_min',
 'B_3_last obs',
 'B_3_last3_avg',
 'B_3_max',
 'B_3_mean',
 'B_3_min',
 'B_40_last obs',
 'B_40_last3_avg',
 'B_40_max',
 'B_40_mean',
 'B_40_min',
 'B_5_last obs',
 'B_5_last3_avg',
 'B_5_max',
 'B_5_mean',
 'B_5_min',
 'B_7_last obs',
 'B_7_last3_avg',
 'B_7_max',
 'B_7_mean',
 'B_7_min',
 'B_8_last obs',
 'B_8_last3_avg',
 'B_8_max',
 'B_8_mean',
 'B_8_min',
 'B_9_last obs',
 'B_9_last3_avg',
 'B_9_max',
 'B_9_mean',
 'B_9_min',
 'D_102_last obs',
 'D_102_last3_avg',
 'D_102_max',
 'D_102_mean',
 'D_102_min',
 'D_103_last obs',
 'D_103_last3_avg',
 'D_103_max',
 'D_103_mean',
 'D_103_min',
 'D_104_last obs',
 'D_104_last3_avg',
 'D_104_max',
 'D_104_mean',
 'D_104_min',
 'D_105_last obs',
 'D_105_last3_avg',
 'D_105_max',
 'D_105_mean',
 'D_105_min',
 'D_106_l

In [29]:
clf = setup(                       train,
            train_size           = 0.7,
            target               = 'target',
            numeric_features     = num_columns,
            ignore_features      = ['customer_ID'],
            fold_strategy        = 'stratifiedkfold',
            fold                 = 3,
            normalize            = True,
            fix_imbalance        = True,
            data_split_stratify  = True,
            session_id           = 42
           )

,Description,Value
0,Session id,42
1,Target,target
2,Target type,Binary
3,Original data shape,"(458913, 811)"
4,Transformed data shape,"(613792, 811)"
5,Transformed train set shape,"(476118, 811)"
6,Transformed test set shape,"(137674, 811)"
7,Ignore features,1
8,Numeric features,550
9,Rows with missing values,100.0%


In [30]:
compare_models(turbo=True, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8985,0.9578,0.8055,0.8033,0.8044,0.7359,0.7359,410.8733
lr,Logistic Regression,0.8914,0.9531,0.8450,0.7618,0.8012,0.7268,0.7287,194.3667
gbc,Gradient Boosting Classifier,0.8893,0.9523,0.8364,0.7600,0.7964,0.7206,0.7221,1801.7533
rf,Random Forest Classifier,0.8856,0.9497,0.8608,0.7400,0.7958,0.7170,0.7210,242.6800
et,Extra Trees Classifier,0.8846,0.9496,0.8583,0.7385,0.7939,0.7144,0.7183,117.4233
ada,Ada Boost Classifier,0.8793,0.9438,0.8216,0.7406,0.7789,0.6962,0.6980,328.5633
knn,K Neighbors Classifier,0.8358,0.8973,0.8418,0.6389,0.7265,0.6123,0.6242,1881.7433
nb,Naive Bayes,0.7708,0.8412,0.9281,0.5342,0.6777,0.5196,0.5676,10.2133
dt,Decision Tree Classifier,0.8321,0.7937,0.7140,0.6634,0.6878,0.5732,0.5740,232.9600
lightgbm,Light Gradient Boosting Machine,0.5977,0.6377,0.5432,0.5278,0.5354,0.4886,0.4887,63.3400


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [31]:
catboost_train_1 = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8983,0.9579,0.8054,0.8025,0.8039,0.7353,0.7353
1,0.8978,0.9574,0.8025,0.8027,0.8026,0.7336,0.7336
2,0.8996,0.9580,0.8086,0.8046,0.8066,0.7388,0.7388
Mean,0.8985,0.9578,0.8055,0.8033,0.8044,0.7359,0.7359
Std,0.0008,0.0003,0.0025,0.0010,0.0017,0.0022,0.0022


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [32]:
save_model(catboost_train_1, 'catboost_train_1')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=C:\Users\srija\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['B_18_last obs', 'B_18_last3_avg', 'B_18_max', 'B_18_mean', 'B_18_min', 'B_2_last obs', 'B_2_last3_avg', 'B_2_max', 'B_2_mean', 'B_2_min', 'B_33_last obs', 'B_33_last3_avg', 'B_33_max', 'B_33_mean', 'B_33_min', 'B_3_last obs',...
                  TransformerWrapper(exclude=None, include=None, transformer=FixImbalancer(estimator=SMOTE(k_neighbors=5, n_jobs=None, random_state=None, sampling_strategy='auto')))),
                 ('normalize',
                  TransformerWrapper(exclude=None, include=None, transformer=StandardScaler(copy=True, with_mean=True, with_std=True))),
                 ['trained_model',
          verbose=False),
 'catboost_train_1.pkl')

In [33]:
catboost_train_1=load_model('catboost_train_1')

Transformation Pipeline and Model Successfully Loaded


In [34]:
feature_imp=pd.DataFrame(list(zip(list(catboost_train_1.steps[-1][1].feature_names_),list(catboost_train_1.steps[-1][1].feature_importances_))),columns=['Feature','Importance']).sort_values(by='Importance',ascending=False)
feature_imp['Importance']=round(feature_imp['Importance'],2)
feature_imp

,Feature,Importance
188,D_54_max,9.66
187,D_54_min,6.99
59,D_112_max,6.48
305,R_27_max,5.44
382,D_44_last obs,5.20
...,...,...
675,D_108_flag_last3,0.00
676,D_109_flag_last3,0.00
677,D_110_flag_last3,0.00
678,D_111_flag_last3,0.00


In [35]:
feature_imp.to_csv('D:\\AMEX\\feature_imp_catboost_train_1_del all.csv')

In [ ]:
test_feat=pd.read_csv('D:\\AMEX\\test_data.processed test data to score_del all.csv')
test_feat

In [ ]:
scored_1 = predict_model(catboost_train_1, data=test_feat)
scored_1

In [ ]:
submission_1 = pd.concat([test_feat.customer_ID,scored_1.Label.rename('prediction')],axis=1)
submission_1

In [ ]:
submission_1.to_csv("D:\\AMEX\\submission_1_del all.csv", index=False)

In [15]:
selected_2=pd.read_csv("D:\\AMEX\\selected_2_del_all.csv")
cols_to_read_2=list(selected_2.selected_2)
cols_to_drop=list(set(train.columns)-set(cols_to_read_2))
cols_to_drop

['D_83_max',
 'D_55_flag_last3',
 'D_124_last3_avg',
 'D_136_max',
 'D_84_mean',
 'D_39_last3_avg',
 'D_72_mean',
 'D_137_last3_avg',
 'D_125_max',
 'D_88_last3_avg',
 'D_135_min',
 'D_70_min',
 'D_71_mean',
 'D_112_last obs',
 'D_102_mean',
 'D_110_flag_last3',
 'D_80_min',
 'D_144_flag_last3',
 'D_124_flag_last3',
 'D_59_flag_last3',
 'D_94_flag_last3',
 'D_139_flag_last',
 'B_5_min',
 'S_3_min',
 'D_51_last3_avg',
 'D_71_min',
 'D_132_last3_avg',
 'B_7_flag_last3',
 'D_134_max',
 'S_23_mean',
 'D_104_flag_last3',
 'D_56_flag_last',
 'D_121_last3_avg',
 'D_58_flag_last3',
 'D_108_flag_last3',
 'D_46_min',
 'D_136_mean',
 'D_87_mean',
 'D_44_flag_last3',
 'D_109_flag_last',
 'D_52_min',
 'D_73_min',
 'D_89_last obs',
 'D_78_flag_last',
 'D_80_last3_avg',
 'D_87_flag_last3',
 'S_3_flag_last3',
 'D_56_last3_avg',
 'D_120_count_4',
 'D_87_last3_avg',
 'D_110_min',
 'D_77_mean',
 'D_142_min',
 'D_46_flag_last',
 'D_53_last3_avg',
 'B_7_min',
 'S_23_min',
 'D_123_last3_avg',
 'B_30_count_6

In [16]:
train_2=train.drop(columns=cols_to_drop)
train_2

,customer_ID,B_8_min,D_103_max,D_112_mean,D_112_max,D_113_min,D_113_max,D_122_min,D_122_max,D_128_max,...,D_80_last obs,D_82_last obs,P_2_last obs,R_2_last obs,R_3_last obs,S_3_last obs,D_54_last3_avg,R_1_last3_avg,R_16_last3_avg,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.000839,1.01000,1.0040,1.010,0.000053,0.009720,0.4300,0.4375,1.00800,...,0.208300,0.5073,0.9346,0.008310,0.000085,0.13500,1.006,0.004963,0.004410,0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.000403,0.00998,1.0060,1.010,0.001209,0.009674,0.2860,0.2952,1.00700,...,0.002120,0.0000,0.8804,0.004980,0.102050,0.16550,1.004,0.006706,0.005863,0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.000247,0.00963,1.0050,1.009,0.000258,0.008410,0.1436,0.1520,0.00920,...,0.000931,0.0000,0.8810,0.001687,0.006264,0.00000,1.006,0.006870,0.002400,0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,1.002000,0.00990,0.6210,1.008,0.201500,0.208000,0.2890,0.4368,1.00800,...,0.005917,0.0000,0.6216,0.004240,0.204300,0.28780,1.003,0.007084,0.172600,0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.001063,1.01000,1.0060,1.010,0.000060,0.009560,0.4290,0.4382,1.00800,...,0.407700,0.0000,0.8720,0.001991,0.000050,0.00000,1.007,0.005272,0.005726,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,ffff41c8a52833b56430603969b9ca48d208e7c192c6a4...,1.000000,1.01000,1.0060,1.010,0.000095,0.009050,0.7190,0.8667,0.00992,...,0.009210,0.0000,0.8440,0.007812,0.710000,0.12870,1.009,0.004940,0.008870,0
458909,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.002172,1.01000,1.0070,1.009,0.001253,0.008320,0.2860,0.2944,1.00800,...,0.200700,0.0000,0.8310,0.009920,0.109130,0.00000,1.005,0.007725,0.172700,0
458910,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,1.001000,1.01000,0.3933,1.009,0.000168,0.009450,0.1442,0.1526,1.00900,...,0.006763,0.0000,0.8003,0.005898,0.008340,0.06665,1.005,0.004913,0.004490,0
458911,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,1.001000,1.01000,1.0050,1.010,0.201200,0.603500,0.1436,0.2952,0.00910,...,0.603500,0.0000,0.7540,0.005466,0.000832,0.40900,1.007,0.006126,0.005573,1


In [17]:
num_columns_2 = sorted([f for f in train_2.columns if f not in a])
len(num_columns_2)

46

In [18]:
clf = setup(                       train_2,
            train_size           = 0.7,
            target               = 'target',
            numeric_features     = num_columns_2,
            ignore_features      = ['customer_ID'],
            fold_strategy        = 'stratifiedkfold',
            fold                 = 3,
            normalize            = True,
            fix_imbalance        = True,
            data_split_stratify  = True,
            session_id           = 42
           )

,Description,Value
0,Session id,42
1,Target,target
2,Target type,Binary
3,Original data shape,"(458913, 47)"
4,Transformed data shape,"(613792, 47)"
5,Transformed train set shape,"(476118, 47)"
6,Transformed test set shape,"(137674, 47)"
7,Ignore features,1
8,Numeric features,46
9,Rows with missing values,97.4%


In [24]:
compare_models(turbo=True, sort='AUC')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.8927,0.9534,0.7921,0.7931,0.7926,0.7203,0.7203,55.4567
lightgbm,Light Gradient Boosting Machine,0.8919,0.9528,0.8031,0.7846,0.7938,0.7206,0.7207,5.6033
xgboost,Extreme Gradient Boosting,0.8911,0.9519,0.7915,0.7887,0.7901,0.7166,0.7166,74.7700
lr,Logistic Regression,0.8757,0.9500,0.8895,0.7064,0.7874,0.7012,0.7105,39.0333
gbc,Gradient Boosting Classifier,0.8870,0.9500,0.8309,0.7565,0.7920,0.7146,0.7161,127.9567
et,Extra Trees Classifier,0.8867,0.9498,0.8448,0.7496,0.7943,0.7166,0.7190,23.1700
rf,Random Forest Classifier,0.8858,0.9485,0.8403,0.7491,0.7921,0.7137,0.7159,47.2267
lda,Linear Discriminant Analysis,0.8685,0.9453,0.8943,0.6899,0.7789,0.6875,0.6993,1.7300
ada,Ada Boost Classifier,0.8767,0.9422,0.8233,0.7333,0.7757,0.6911,0.6933,24.7067
nb,Naive Bayes,0.8640,0.9257,0.7646,0.7252,0.7444,0.6519,0.6523,1.3467


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [19]:
catboost_train_2 = create_model('catboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8931,0.9537,0.7914,0.7948,0.7931,0.7210,0.7210
1,0.8923,0.9530,0.7922,0.7919,0.7920,0.7193,0.7193
2,0.8927,0.9535,0.7929,0.7927,0.7928,0.7204,0.7204
Mean,0.8927,0.9534,0.7921,0.7931,0.7926,0.7203,0.7203
Std,0.0003,0.0003,0.0006,0.0012,0.0004,0.0007,0.0007


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
save_model(catboost_train_2, 'catboost_train_2')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=C:\Users\srija\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['B_33_last obs', 'B_3_last obs', 'B_7_last obs', 'B_8_min', 'B_9_last obs', 'D_103_max', 'D_112_max', 'D_112_mean', 'D_113_max', 'D_113_min', 'D_122_max', 'D_122_min', 'D_128_max', 'D_143_max', 'D_39_last obs', 'D_41_last obs', '...
                  TransformerWrapper(exclude=None, include=None, transformer=FixImbalancer(estimator=SMOTE(k_neighbors=5, n_jobs=None, random_state=None, sampling_strategy='auto')))),
                 ('normalize',
                  TransformerWrapper(exclude=None, include=None, transformer=StandardScaler(copy=True, with_mean=True, with_std=True))),
                 ['trained_model',
          verbose=False),
 'catboost_train_2.pkl')

In [21]:
lightgbm_train_2 = create_model('lightgbm')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8923,0.9531,0.8029,0.7859,0.7943,0.7214,0.7215
1,0.8912,0.9523,0.8023,0.7828,0.7924,0.7187,0.7188
2,0.8923,0.9529,0.8042,0.7852,0.7946,0.7216,0.7217
Mean,0.8919,0.9528,0.8031,0.7846,0.7938,0.7206,0.7207
Std,0.0006,0.0003,0.0008,0.0013,0.0010,0.0013,0.0013


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
save_model(lightgbm_train_2, 'lightgbm_train_2')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=C:\Users\srija\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['B_33_last obs', 'B_3_last obs', 'B_7_last obs', 'B_8_min', 'B_9_last obs', 'D_103_max', 'D_112_max', 'D_112_mean', 'D_113_max', 'D_113_min', 'D_122_max', 'D_122_min', 'D_128_max', 'D_143_max', 'D_39_last obs', 'D_41_last obs', '...
                  LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                 colsample_bytree=1.0, importance_type='split',
                                 learning_rate=0.1, max_depth=-1,
                                 min_child_samples=20, min_child_weight=0.001,
                                 min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                 num_leaves=31, objective=None, random_state=42,
                                 reg_alpha=0.0, reg_lambda=0.0, silent='warn',
                                 subsample=1.0, sub

In [23]:
xgboost_train_2=create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8911,0.9522,0.7920,0.7884,0.7902,0.7167,0.7167
1,0.8909,0.9518,0.7907,0.7886,0.7896,0.7160,0.7160
2,0.8913,0.9518,0.7919,0.7892,0.7905,0.7172,0.7172
Mean,0.8911,0.9519,0.7915,0.7887,0.7901,0.7166,0.7166
Std,0.0002,0.0002,0.0006,0.0003,0.0004,0.0005,0.0005


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [24]:
save_model(xgboost_train_2, 'xgboost_train_2')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=C:\Users\srija\AppData\Local\Temp\joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None, include=['B_33_last obs', 'B_3_last obs', 'B_7_last obs', 'B_8_min', 'B_9_last obs', 'D_103_max', 'D_112_max', 'D_112_mean', 'D_113_max', 'D_113_min', 'D_122_max', 'D_122_min', 'D_128_max', 'D_143_max', 'D_39_last obs', 'D_41_last obs', '...
                                gamma=0, gpu_id=-1, grow_policy='depthwise',
                                importance_type=None, interaction_constraints='',
                                learning_rate=0.300000012, max_bin=256,
                                max_cat_to_onehot=4, max_delta_step=0,
                                max_depth=6, max_leaves=0, min_child_weight=1,
                                missing=nan, monotone_constraints='()',
                                n_estimators=100, n_jobs=-1, num_parallel_tree=1,
                                objective='binary:logisti

In [25]:
test_feat=pd.read_csv('D:\\AMEX\\test_del_all to score.csv')
test_feat

,customer_ID,B_18_mean,B_18_min,B_18_max,B_2_mean,B_2_min,B_2_max,B_3_mean,B_3_min,B_3_max,...,D_120_count_9,D_120_count_10,D_120_count_11,D_120_count_12,D_120_count_13,D_120_last_0.0,D_120_last_1.0,D_120_nunique_0,D_120_nunique_1,D_120_nunique_2
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.59100,0.5854,0.59330,0.89840,0.81050,1.01000,0.009735,0.002348,0.020000,...,0,0,0,0,0,0,1,0,1,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.00500,1.0010,1.00900,1.00400,1.00100,1.00900,0.006160,0.001596,0.011635,...,0,0,0,0,1,1,0,0,1,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.74850,0.5845,1.01000,0.90140,0.81000,1.01000,0.008550,0.000155,0.015950,...,0,0,0,0,1,1,0,0,1,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.14500,0.1362,0.16560,0.09090,0.03268,0.20570,0.667000,0.389200,0.961000,...,0,0,0,0,1,1,0,0,0,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.21030,0.2056,0.21330,0.04193,0.02017,0.10675,0.616700,0.350600,0.889600,...,0,0,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.84670,0.6480,1.00700,0.81540,0.81100,0.81930,0.024120,0.020870,0.028900,...,0,0,0,0,1,1,0,0,1,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.43680,0.1863,1.00600,0.41160,0.08250,1.01000,0.037570,0.007572,0.070500,...,0,0,0,0,1,1,0,0,0,1
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.53170,0.1511,0.59230,0.90230,0.17110,1.01000,0.028580,0.002672,0.079470,...,0,0,0,0,1,1,0,0,0,1
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.01813,0.0098,0.08746,0.13490,0.10333,0.19290,0.119260,0.099900,0.132200,...,0,0,0,0,1,0,1,0,0,1


In [26]:
cols_to_drop

['D_83_max',
 'D_55_flag_last3',
 'D_124_last3_avg',
 'D_136_max',
 'D_84_mean',
 'D_39_last3_avg',
 'D_72_mean',
 'D_137_last3_avg',
 'D_125_max',
 'D_88_last3_avg',
 'D_135_min',
 'D_70_min',
 'D_71_mean',
 'D_112_last obs',
 'D_102_mean',
 'D_110_flag_last3',
 'D_80_min',
 'D_144_flag_last3',
 'D_124_flag_last3',
 'D_59_flag_last3',
 'D_94_flag_last3',
 'D_139_flag_last',
 'B_5_min',
 'S_3_min',
 'D_51_last3_avg',
 'D_71_min',
 'D_132_last3_avg',
 'B_7_flag_last3',
 'D_134_max',
 'S_23_mean',
 'D_104_flag_last3',
 'D_56_flag_last',
 'D_121_last3_avg',
 'D_58_flag_last3',
 'D_108_flag_last3',
 'D_46_min',
 'D_136_mean',
 'D_87_mean',
 'D_44_flag_last3',
 'D_109_flag_last',
 'D_52_min',
 'D_73_min',
 'D_89_last obs',
 'D_78_flag_last',
 'D_80_last3_avg',
 'D_87_flag_last3',
 'S_3_flag_last3',
 'D_56_last3_avg',
 'D_120_count_4',
 'D_87_last3_avg',
 'D_110_min',
 'D_77_mean',
 'D_142_min',
 'D_46_flag_last',
 'D_53_last3_avg',
 'B_7_min',
 'S_23_min',
 'D_123_last3_avg',
 'B_30_count_6

In [27]:
test_2=test_feat.drop(columns=cols_to_drop)
test_2

,customer_ID,B_8_min,D_103_max,D_112_mean,D_112_max,D_113_min,D_113_max,D_122_min,D_122_max,D_128_max,...,D_70_last obs,D_80_last obs,D_82_last obs,P_2_last obs,R_2_last obs,R_3_last obs,S_3_last obs,D_54_last3_avg,R_1_last3_avg,R_16_last3_avg
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,1.003000,0.009280,1.004000,1.00700,0.002960,0.009605,0.1433,0.1492,0.008780,...,0.006584,0.006065,0.5030,0.5690,0.007510,0.109440,0.1494,1.0070,0.007305,0.004333
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,1.001000,0.006363,1.005000,1.01000,0.000318,0.008770,0.5720,0.5815,1.009000,...,0.003786,0.001574,0.0000,0.8413,0.002270,0.008150,0.1122,1.0030,0.005886,0.005253
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,1.000000,1.010000,0.854000,1.00900,0.401000,0.410000,0.2856,0.2947,0.009290,...,0.258300,0.000137,0.5083,0.6978,0.000715,0.001311,0.1661,1.0050,0.004190,0.003534
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.000242,1.010000,0.012540,0.02113,0.201500,0.209100,0.1437,0.1526,0.009880,...,0.501500,0.404000,1.0000,0.5130,0.009120,0.104200,0.1812,1.0030,0.002186,0.004020
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1.000000,0.009970,1.005000,1.01000,0.000737,0.007626,0.1442,0.1525,1.009000,...,0.006416,0.007603,0.0000,0.2544,0.007126,0.004543,0.1683,1.0060,0.337400,0.005287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.000352,0.009880,1.006000,1.01000,0.000248,0.009560,0.5723,0.5810,1.008000,...,0.007740,0.200700,0.0000,0.6470,0.006280,0.006640,0.1630,1.0060,0.005592,0.003883
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,1.001000,1.010000,1.006000,1.00900,0.000698,0.209400,0.2866,0.2930,1.008000,...,0.253200,0.209500,0.0000,0.4712,0.009575,0.508300,0.8570,1.0050,0.339800,0.505400
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,1.001000,0.009920,0.007164,0.01476,0.000125,0.009980,0.5723,0.5815,1.010000,...,0.001062,0.000159,0.5024,0.2064,0.003794,0.408400,0.3210,0.3384,0.007640,0.172000
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,1.000000,1.010000,0.928000,1.01000,0.000622,0.009820,0.1439,0.4333,0.008865,...,0.508000,0.407500,0.0000,0.5710,0.001982,0.005043,0.2308,1.0030,0.006027,0.007380


In [28]:
scored_2_catboost = predict_model(catboost_train_2, data=test_2)
scored_2_lgbm = predict_model(lightgbm_train_2, data=test_2)
scored_2_xgboost = predict_model(xgboost_train_2, data=test_2)

In [29]:
submission_2_catboost = pd.concat([test_2.customer_ID,scored_2_catboost.Label.rename('prediction')],axis=1)
submission_2_lgbm = pd.concat([test_2.customer_ID,scored_2_lgbm.Label.rename('prediction')],axis=1)
submission_2_xgboost = pd.concat([test_2.customer_ID,scored_2_xgboost.Label.rename('prediction')],axis=1)

In [30]:
submission_2_catboost

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,1
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,1
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [31]:
submission_2_lgbm

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,1
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,1
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [32]:
submission_2_xgboost

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,1
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,1
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,1
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,1
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [33]:
submission_2_catboost.to_csv("D:\\AMEX\\submission_2_catboost.csv", index=False)
submission_2_lgbm.to_csv("D:\\AMEX\\submission_2_lgbm.csv", index=False)
submission_2_xgboost.to_csv("D:\\AMEX\\submission_2_xgboost.csv", index=False)